In [22]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
import tldextract
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [23]:
# Загрузка данных
data = pd.read_csv("Malicious-URLs.csv")

# Удаление дубликатов и пропусков
data = data.drop_duplicates(subset="url").dropna()

# Проверка баланса классов
print("Распределение классов:")
print(data["label"].value_counts())

Распределение классов:
label
benign        10991
defacement     2727
phishing        912
malware         303
Name: count, dtype: int64


In [24]:
data["label"] = data["label"].map({'defacement': 0, 'phishing': 1, 'benign': 2, 'malware': 3})
data

,count,url,label
0,0,br-icloud.com.br,1
1,1,mp3raid.com/music/krizz_kaliko.html,2
2,2,bopsecrets.org/rexroth/cr/1.htm,2
3,3,http://www.garage-pirenne.be/index.php?option=...,0
4,4,http://adventure-nicaragua.net/index.php?optio...,0
...,...,...,...
14995,14995,fortdefianceind.com/,2
14996,14996,bookrags.com/lens/go.php?u=Hannah_Webster_Foster,2
14997,14997,burbankairport.com/parking/buses-trains.html,2
14998,14998,http://otomoto.pl/oferta/subaru-outback-legacy...,2


In [25]:
def extract_features(url):
    features = {}
    try:
        # Лексические признаки
        features["url_length"] = len(url)
        features["num_special_chars"] = sum(url.count(c) for c in ['@', '%', '//', '?', '='])
        
        # Анализ домена
        ext = tldextract.extract(url)
        features["subdomain_count"] = len(ext.subdomain.split('.')) if ext.subdomain else 0
        features["domain_length"] = len(ext.domain)
        features["has_ip"] = 1 if any(part.isdigit() for part in ext.domain.split('.')) else 0
        
        # Семантические признаки
        keywords = ['login', 'admin', 'exe', 'php', 'config', 'secure']
        features["keyword_count"] = sum(1 for kw in keywords if kw in url)
        return features
    except Exception as e:
        print(f"Ошибка обработки URL {url}: {e}")
        return None

# Применение функции
features = data["url"].apply(extract_features)
valid_indices = features[features.notna()].index
data = data.loc[valid_indices]
features = features.loc[valid_indices]

# Создание DataFrame с признаками
features_df = pd.DataFrame(features.tolist())
labels = data["label"].values

In [26]:
# ----------------------------------------------------------
# Шаг 4: Обработка дисбаланса
# ----------------------------------------------------------

# Проверяем уникальные метки
classes = np.unique(labels)
print("Уникальные классы в данных:", classes)

if len(classes) < 2:
    raise ValueError("Данные содержат только один класс! Необходимы как минимум два класса для классификации.")

# Рассчитываем веса классов
class_weights = compute_class_weight("balanced", classes=classes, y=labels)
class_weights_dict = {cls: weight for cls, weight in zip(classes, class_weights)}

# Для XGBoost и LightGBM (только для бинарной классификации)
if len(classes) == 2:
    # Определяем индекс положительного класса (обычно 1)
    positive_class = 1 if 1 in classes else classes[-1]
    scale_pos_weight = class_weights_dict[positive_class] / class_weights_dict[classes[0]]
    print(f"\nВесовой коэффициент для класса {positive_class}: {scale_pos_weight:.1f}")
else:
    scale_pos_weight = None  # Для многоклассовой классификации не используется

Уникальные классы в данных: [0 1 2 3]


In [27]:
X_train, X_test, y_train, y_test = train_test_split(
    features_df, 
    labels, 
    test_size=0.2, 
    stratify=labels, 
    random_state=42
)

In [28]:
# Инициализация моделей
xgb_model = XGBClassifier(
    scale_pos_weight=scale_pos_weight,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    random_state=42
)

rf_model = RandomForestClassifier(
    class_weight=class_weights_dict,
    n_estimators=100,
    max_depth=10,
    random_state=42
)

lgbm_model = LGBMClassifier(
    class_weight=class_weights_dict,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    random_state=42
)

# Создание ансамбля (мягкое голосование)
ensemble = VotingClassifier(
    estimators=[
        ("xgb", xgb_model),
        ("rf", rf_model),
        ("lgbm", lgbm_model)
    ],
    voting="soft"  # Используем вероятности классов
)

# Обучение ансамбля
ensemble.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000426 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 270
[LightGBM] [Info] Number of data points in the train set: 11946, number of used features: 6
[LightGBM] [Info] Start training from score -1.385859
[LightGBM] [Info] Start training from score -1.385494
[LightGBM] [Info] Start training from score -1.386133
[LightGBM] [Info] Start training from score -1.387693
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None,
                                            feature_weights=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constra...
                             ('rf',
                              RandomForestClassifier(class_weight={0: 1.368995232856619,
                                                                   1: 4.093475877192983,
                                                                   2: 0.3396642707669912,
                                                                   3: 12.320957095709572},
                                                     max_depth=10,
                                                     random_state=42)),
                             ('lgbm',
                              LGBMClassifier(class_weight={0: 1.368995232856619,
                                                           1: 4.093475877192983,
                                                           2: 0.3396642707669912,
                                                           3: 12.320957095709572},
                                             learning_rate=0.05, max_depth=6,
                                             random_state=42, subsample=0.8))],
                 voting='soft')

In [30]:
# Предсказание
y_pred = ensemble.predict(X_test)
y_proba = ensemble.predict_proba(X_test)[:, 1]  # Вероятность класса 1

# Метрики
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

#print(f"\nROC-AUC: {roc_auc_score(y_test, y_proba):.2f}")

# Важность признаков (на примере XGBoost)
xgb_model.fit(X_train, y_train)  # Переобучаем XGBoost отдельно для анализа
plt.figure(figsize=(10, 6))
xgb.plot_importance(xgb_model, importance_type="gain")
plt.title("Важность признаков (XGBoost)")
plt.show()


Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.92      0.88       545
           1       0.42      0.74      0.54       182
           2       0.98      0.89      0.93      2199
           3       0.46      0.57      0.51        61

    accuracy                           0.88      2987
   macro avg       0.68      0.78      0.72      2987
weighted avg       0.91      0.88      0.89      2987



NameError: name 'xgb' is not defined

<Figure size 1000x600 with 0 Axes>

In [34]:
def predict_url(url: str, model, threshold: float = 0.5) -> str:
    try:
        # Извлечение признаков
        features = extract_features(url)
        if not features:
            return "Ошибка извлечения признаков."
        
        # Преобразование в DataFrame
        features_df = pd.DataFrame([features])
        
        # Предсказание
        proba = model.predict_proba(features_df)[0][1]
        return "Вредоносный" if proba > threshold else "Доброкачественный"
    except Exception as e:
        return f"Ошибка: {str(e)}"

# Пример использования
print(predict_url("http://fsfll.fgawudownsyfuf.info", ensemble, threshold=0.7))  # Вредоносный
print(predict_url("https://google.com", ensemble))                   # Доброкачественный

Доброкачественный
Доброкачественный


In [32]:
from sklearn.metrics import precision_recall_curve

# Поиск оптимального порога через Precision-Recall
precisions, recalls, thresholds = precision_recall_curve(y_test, y_proba)
best_threshold = thresholds[np.argmax(precisions >= 0.9)]  # Порог для 90% точности

print(f"\nОптимальный порог: {best_threshold:.2f}")

ValueError: multiclass format is not supported